In [57]:
import pandas as pd
import numpy as np

df_knn = pd.read_csv('KNN_resultados.csv')
df_xgb = pd.read_csv('XGBoost_resultados.csv')
df_ARIMA = pd.read_csv('ARIMA_resultados.csv')

In [58]:
df_knn = df_knn.drop(columns=["Unnamed: 0", "Overfitting?", "Underfitting?"])
df_knn = df_knn.rename(columns={
    'Real':'Valores Reais',
    'Previsto':'knn_Previsto',
    'Erro Absoluto':'knn_MAE',
    'Train RMSE':'knn_train_RMSE',
    'Test RMSE':'knn_test_RMSE'
        })

In [59]:
df_xgb = df_xgb.drop(columns=["Unnamed: 0", "Overfitting?", "Underfitting?"])
df_xgb = df_xgb.rename(columns={
    'Real':'Valores Reais',
    'Previsto':'xgb_Previsto',
    'Erro Absoluto':'xgb_MAE',
    'Train RMSE':'xgb_train_RMSE',
    'Test RMSE':'xgb_test_RMSE'
        })

In [60]:
df_ARIMA = df_ARIMA.drop(columns=["Unnamed: 0", "Overfitting?", "Underfitting?"])
df_ARIMA = df_ARIMA.rename(columns={
    'Real':'Valores Reais',
    'Previsto':'ARIMA_Previsto',
    'Erro Absoluto':'ARIMA_MAE',
    'Train RMSE':'ARIMA_train_RMSE',
    'Test RMSE':'ARIMA_test_RMSE'
        })

In [61]:
df = pd.merge(df_knn, df_xgb, on=['Empresa', 'Data', 'Valores Reais'], how='right')
df = pd.merge(df, df_ARIMA, on=['Empresa', 'Data', 'Valores Reais'], how='right')

In [62]:
from sklearn.metrics import mean_squared_error
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Função para calcular RMSE
def calcular_rmse_knn(df):
    return np.sqrt(mean_squared_error(df['Valores Reais'], df['knn_Previsto']))

# Aplicar por grupo (por empresa)
rmse_por_empresa_knn = df.groupby('Empresa').apply(calcular_rmse_knn).reset_index()
rmse_por_empresa_knn.columns = ['Empresa', 'RMSE_knn']

In [63]:
# Função para calcular RMSE
def calcular_rmse_xgb(df):
    return np.sqrt(mean_squared_error(df['Valores Reais'], df['xgb_Previsto']))

# Aplicar por grupo (por empresa)
rmse_por_empresa_xgb = df.groupby('Empresa').apply(calcular_rmse_xgb).reset_index()
rmse_por_empresa_xgb.columns = ['Empresa', 'RMSE_xgb']

In [64]:
# Função para calcular RMSE
def calcular_rmse_ARIMA(df):
    return np.sqrt(mean_squared_error(df['Valores Reais'], df['ARIMA_Previsto']))

# Aplicar por grupo (por empresa)
rmse_por_empresa_ARIMA = df.groupby('Empresa').apply(calcular_rmse_ARIMA).reset_index()
rmse_por_empresa_ARIMA.columns = ['Empresa', 'RMSE_ARIMA']

In [65]:
rmse_final = pd.merge(rmse_por_empresa_knn, rmse_por_empresa_xgb, on=['Empresa'], how='right')
rmse_final = pd.merge(rmse_final, rmse_por_empresa_ARIMA, on=['Empresa'], how='right')

empresa_maior_rmse_knn = rmse_final.loc[rmse_final['RMSE_knn'].idxmax()]
empresa_menor_rmse_knn = rmse_final.loc[rmse_final['RMSE_knn'].idxmin()]

# Maior e menor RMSE para o XGB
empresa_maior_rmse_xgb = rmse_final.loc[rmse_final['RMSE_xgb'].idxmax()]
empresa_menor_rmse_xgb = rmse_final.loc[rmse_final['RMSE_xgb'].idxmin()]

# Maior e menor RMSE para o ARIMA
empresa_maior_rmse_ARIMA = rmse_final.loc[rmse_final['RMSE_ARIMA'].idxmax()]
empresa_menor_rmse_ARIMA = rmse_final.loc[rmse_final['RMSE_ARIMA'].idxmin()]

print("Maior RMSE KNN:", empresa_maior_rmse_knn)
print("Menor RMSE KNN:", empresa_menor_rmse_knn)

print("Maior RMSE XGB:", empresa_maior_rmse_xgb)
print("Menor RMSE XGB:", empresa_menor_rmse_xgb)

print("Maior RMSE ARIMA:", empresa_maior_rmse_ARIMA)
print("Menor RMSE ARIMA:", empresa_menor_rmse_ARIMA)

Maior RMSE KNN: Empresa       Petrobras
RMSE_knn       0.442604
RMSE_xgb       0.324413
RMSE_ARIMA     0.770676
Name: 193, dtype: object
Menor RMSE KNN: Empresa       Dtcom Direct
RMSE_knn          0.000028
RMSE_xgb          0.000027
RMSE_ARIMA        0.000039
Name: 78, dtype: object
Maior RMSE XGB: Empresa       Petrobras
RMSE_knn       0.442604
RMSE_xgb       0.324413
RMSE_ARIMA     0.770676
Name: 193, dtype: object
Menor RMSE XGB: Empresa       Dtcom Direct
RMSE_knn          0.000028
RMSE_xgb          0.000027
RMSE_ARIMA        0.000039
Name: 78, dtype: object
Maior RMSE ARIMA: Empresa       Petrobras
RMSE_knn       0.442604
RMSE_xgb       0.324413
RMSE_ARIMA     0.770676
Name: 193, dtype: object
Menor RMSE ARIMA: Empresa       Recrusul
RMSE_knn      0.000058
RMSE_xgb      0.000058
RMSE_ARIMA    0.000022
Name: 210, dtype: object


In [66]:
grupo = df.groupby('Empresa')[['knn_MAE', 'knn_train_RMSE', 'knn_test_RMSE', 'xgb_MAE', 'xgb_train_RMSE', 'xgb_test_RMSE', 'ARIMA_MAE', 'ARIMA_train_RMSE', 'ARIMA_test_RMSE']].mean().reset_index()
grupo #media do MAE, media do RMSE de treino e media do RMSE teste por modelo

,Empresa,knn_MAE,knn_train_RMSE,knn_test_RMSE,xgb_MAE,xgb_train_RMSE,xgb_test_RMSE,ARIMA_MAE,ARIMA_train_RMSE,ARIMA_test_RMSE
0,3tentos,0.005229,0.005851,0.005229,0.006244,0.008663,0.006244,0.005168,0.009944,0.005168
1,Aco Altona,0.000354,0.000281,0.000354,0.000361,0.000370,0.000361,0.000574,0.000579,0.000574
2,Aeris,0.003211,0.004726,0.003211,0.004010,0.006807,0.004010,0.003524,0.006561,0.003524
3,Afluente T,0.000388,0.000294,0.000388,0.000364,0.000344,0.000364,0.000543,0.000327,0.000543
4,Agrogalaxy,0.008917,0.007636,0.008917,0.012790,0.011182,0.012790,0.013759,0.013126,0.013759
...,...,...,...,...,...,...,...,...,...,...
275,Wilson Sons,0.001594,0.001086,0.001594,0.002507,0.001897,0.002507,0.002549,0.002134,0.002549
276,Wiz Co,0.006316,0.004089,0.006316,0.006547,0.006479,0.006547,0.008405,0.006001,0.008405
277,Wlm Ind Com,0.000201,0.000173,0.000201,0.000181,0.000202,0.000181,0.000208,0.000234,0.000208
278,Yduqs Part,0.005485,0.005962,0.005485,0.008619,0.011299,0.008619,0.007674,0.010225,0.007674


In [67]:
knn_MAE_media = df['knn_MAE'].mean()
knn_train_RMSE_media = df['knn_train_RMSE'].mean()
knn_test_RMSE_media = grupo['knn_test_RMSE'].mean()

xgb_MAE_media = df['xgb_MAE'].mean()
xgb_train_RMSE_media = df['xgb_train_RMSE'].mean()
xgb_test_RMSE_media = df['xgb_test_RMSE'].mean()

ARIMA_MAE_media = df['ARIMA_MAE'].mean()
ARIMA_train_RMSE_media = df['ARIMA_train_RMSE'].mean()
ARIMA_test_RMSE_media = df['ARIMA_test_RMSE'].mean()

print(f'MAE Médio KNN: {knn_MAE_media}') #é diferente do MAE geral do modelo pois as médias de cada empresas possuem ns diferentes (datasets com observações diferentes)
print(f'RMSE Treino Médio KNN: {knn_train_RMSE_media}')
print(f'RMSE Teste Médio KNN: {knn_test_RMSE_media}')

print(f'MAE Médio XGB: {xgb_MAE_media}')
print(f'RMSE Treino Médio XGB: {xgb_train_RMSE_media}')
print(f'RMSE Teste Médio XGB: {xgb_test_RMSE_media}')

print(f'MAE Médio ARIMA: {ARIMA_MAE_media}')
print(f'RMSE Treino Médio ARIMA: {ARIMA_train_RMSE_media}')
print(f'RMSE Teste Médio ARIMA: {ARIMA_test_RMSE_media}')

MAE Médio KNN: 0.009458392368906266
RMSE Treino Médio KNN: 0.00993908888274029
RMSE Teste Médio KNN: 0.008573297003121443
MAE Médio XGB: 0.013974778733443025
RMSE Treino Médio XGB: 0.014617835929099474
RMSE Teste Médio XGB: 0.013974778733443025
MAE Médio ARIMA: 0.01621570211526307
RMSE Treino Médio ARIMA: 0.01853541382981382
RMSE Teste Médio ARIMA: 0.01621570211526307


In [68]:
knn_MAE_std = df['knn_MAE'].std()
knn_train_RMSE_std = df['knn_train_RMSE'].std()
knn_test_RMSE_std = grupo['knn_test_RMSE'].std()

xgb_MAE_std = df['xgb_MAE'].std()
xgb_train_RMSE_std = df['xgb_train_RMSE'].std()
xgb_test_RMSE_std = df['xgb_test_RMSE'].std()

ARIMA_MAE_std = df['ARIMA_MAE'].std()
ARIMA_train_RMSE_std = df['ARIMA_train_RMSE'].std()
ARIMA_test_RMSE_std = df['ARIMA_test_RMSE'].std()

print(f'MAE Std KNN: {knn_MAE_std}') #é diferente do MAE geral do modelo pois as médias de cada empresas possuem ns diferentes (datasets com observações diferentes)
print(f'RMSE Treino Std KNN: {knn_train_RMSE_std}')
print(f'RMSE Teste Std KNN: {knn_test_RMSE_std}')

print(f'MAE Std XGB: {xgb_MAE_std}')
print(f'RMSE Treino Std XGB: {xgb_train_RMSE_std}')
print(f'RMSE Teste Std XGB: {xgb_test_RMSE_std}')

print(f'MAE Std ARIMA: {ARIMA_MAE_std}')
print(f'RMSE Treino Std ARIMA: {ARIMA_train_RMSE_std}')
print(f'RMSE Teste Std ARIMA: {ARIMA_test_RMSE_std}')

MAE Std KNN: 0.0398768713513208
RMSE Treino Std KNN: 0.04726022671238258
RMSE Teste Std KNN: 0.019966259284070597
MAE Std XGB: 0.04114984947850017
RMSE Treino Std XGB: 0.03042833801038499
RMSE Teste Std XGB: 0.04114984947850017
MAE Std ARIMA: 0.06632561698895731
RMSE Treino Std ARIMA: 0.07881102829236243
RMSE Teste Std ARIMA: 0.06632561698895731


In [69]:
from scipy.stats import friedmanchisquare
from scipy.stats import rankdata

friedman_MAE = friedmanchisquare(grupo['knn_MAE'], grupo['xgb_MAE'], grupo['ARIMA_MAE']) 

friedman_MAE

FriedmanchisquareResult(statistic=253.4000000000001, pvalue=9.438199279455322e-56)

In [70]:
# Aplicar ranking por linha (menor MAE = melhor)
ranks_MAE = grupo[['knn_MAE', 'xgb_MAE', 'ARIMA_MAE']].apply(rankdata, axis=1)
mean_ranks_MAE = ranks_MAE.mean(axis=0)

# Exibir ranking médio
print("Rank médio da MAE dos modelos:")
for i, rank in enumerate(mean_ranks_MAE, start=1):
    print(f"Modelo {i}: {rank:.3f}")

Rank médio da MAE dos modelos:
Modelo 1: 1.250
Modelo 2: 2.200
Modelo 3: 2.550


In [71]:
friedman_RMSE_train = friedmanchisquare(grupo['knn_train_RMSE'], grupo['xgb_train_RMSE'], grupo['ARIMA_train_RMSE']) 

friedman_RMSE_train

FriedmanchisquareResult(statistic=409.93208221626446, pvalue=9.646713248570883e-90)

In [72]:
# Aplicar ranking por linha (menor MAE = melhor)
ranks_RMSE_train = grupo[['knn_train_RMSE', 'xgb_train_RMSE', 'ARIMA_train_RMSE']].apply(rankdata, axis=1)
mean_ranks_RMSE_train = ranks_RMSE_train.mean(axis=0)

# Exibir ranking médio
print("Rank médio da RMSE do treino dos modelos:")
for i, rank in enumerate(mean_ranks_RMSE_train, start=1):
    print(f"Modelo {i}: {rank:.3f}")

Rank médio da RMSE do treino dos modelos:
Modelo 1: 1.020
Modelo 2: 2.388
Modelo 3: 2.593
